In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size and path to the dataset
img_size = (512, 512)
path = "D:\\Casper\\OTHER\\Data\\dpaml_hw3\\face_data"

# Define different data augmentations
datagen = ImageDataGenerator(rescale=1./255)
datagen_flip_v = ImageDataGenerator(
    rescale=1./255,
    vertical_flip=True
)
datagen_flip_h = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True
    
)
datagen_flip_both = ImageDataGenerator(
    rescale=1./255,
    vertical_flip=True,
    horizontal_flip=True
)

# Create data generators
data_flow = datagen.flow_from_directory(path, target_size=img_size, class_mode='categorical', batch_size=1, shuffle=True)
data_flow_v = datagen_flip_v.flow_from_directory(path, target_size=img_size, class_mode='categorical', batch_size=1, shuffle=True)
data_flow_h = datagen_flip_h.flow_from_directory(path, target_size=img_size, class_mode='categorical', batch_size=1, shuffle=True)
data_flow_both = datagen_flip_both.flow_from_directory(path, target_size=img_size, class_mode='categorical', batch_size=1, shuffle=True)
print(data_flow.batch_size)


c:\Users\User\anaconda3\envs\tf37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\User\anaconda3\envs\tf37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\User\anaconda3\envs\tf37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\User\anaconda3\envs\tf37\lib\site-packages\tensorflow\python\framework

Found 310 images belonging to 4 classes.
Found 310 images belonging to 4 classes.
Found 310 images belonging to 4 classes.
Found 310 images belonging to 4 classes.
1


In [2]:
from itertools import cycle
import random
import numpy as np
def combined_generator(*generators):
    while True:
        for g in generators:
            yield next(g)

combined_gen_test = combined_generator(data_flow, data_flow_v, data_flow_h, data_flow_both)


def combined_shuffled_generator(batch_size, *generators):
    # Create an iterator pool from all generators
    iterator_pool = [iter(gen) for gen in generators]
    
    while True:
        # Prepare to collect a batch of the specified size
        combined_batch_x = []
        combined_batch_y = []
        
        for _ in range(batch_size):
            # Select a generator at random
            gen_iter = random.choice(iterator_pool)
            try:
                x, y = next(gen_iter)
                combined_batch_x.append(x)
                combined_batch_y.append(y)
            except StopIteration:
                # Handle the case where a generator is exhausted if not looping indefinitely
                continue

        # Yield the combined batch
        yield np.vstack(combined_batch_x), np.vstack(combined_batch_y)

# Example usage
combined_gen = combined_shuffled_generator(4, data_flow, data_flow_v, data_flow_h, data_flow_both)

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models, Model

class SimpleCNN_a(tf.keras.Model):
    def __init__(self):
        super(SimpleCNN_a, self).__init__()
        self.conv1 = layers.Conv2D(20, (3, 3), strides=2)
        # Pooling is commented out in the original PyTorch code, we'll skip it here too
        self.conv2 = layers.Conv2D(40, (3, 3), strides=2)
        self.conv3 = layers.Conv2D(20, (3, 3), strides=2)
        self.conv4 = layers.Conv2D(1, (3, 3), strides=2)
        self.flatten = layers.Flatten()
        # Inferred from PyTorch code - assuming the feature size here, adjust as needed
        self.fc = layers.Dense(4)

    def call(self, x):
        x = self.conv1(x)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = tf.nn.relu(x)
        x = self.conv3(x)
        x = tf.nn.relu(x)
        x = self.conv4(x)
        x = tf.nn.relu(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

class SimpleCNN_b(tf.keras.Model):
    def __init__(self):
        super(SimpleCNN_b, self).__init__()
        self.conv1 = layers.Conv2D(50, (3, 3), strides=2)
        self.pool = layers.MaxPooling2D(2)
        self.conv2 = layers.Conv2D(1, (3, 3), strides=2)
        self.flatten = layers.Flatten()
        self.fc = layers.Dense(4)

    def call(self, x):
        x = self.conv1(x)
        x = tf.nn.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = tf.nn.relu(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

# Creating an instance of the second model and testing it
model = SimpleCNN_b()

# Creating a random input tensor
img = tf.random.normal([2, 512, 512, 3])  # Note TensorFlow uses NHWC format by default
output = model(img)
print(output.shape)  # Should print (2, 4)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
(2, 4)


In [7]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import time

# Define the MobileNet model
class_num = data_flow.num_classes  # Assuming class numbers are derived from the generator
print(class_num)
model = MobileNet(input_shape=(512, 512, 3), weights=None, classes=class_num)
model = SimpleCNN_b()
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
# Calculate the steps per epoch

# Start trainin
model.fit(combined_gen_test, epochs=5, steps_per_epoch=310)
model.save('tf_simpleCNN', save_format='tf')

# Later, load the model from this directory
model = tf.keras.models.load_model('my_model_saved_model_format')


4


Epoch 1/5
310/310 [==============================] - 8s 26ms/step - loss: 7.6801 - acc: 0.2881
Epoch 2/5
310/310 [==============================] - 8s 25ms/step - loss: 7.9031 - acc: 0.3333
Epoch 3/5
310/310 [==============================] - 8s 24ms/step - loss: 7.7991 - acc: 0.3021
Epoch 4/5
310/310 [==============================] - 8s 24ms/step - loss: 7.9031 - acc: 0.3378
Epoch 5/5
310/310 [==============================] - 7s 24ms/step - loss: 8.2150 - acc: 0.3186


NotImplementedError: Saving the model as SavedModel is not supported in TensorFlow 1.Xgraph mode. Please enable eager execution or use the "h5" save format.

In [11]:
model.save_weights('tf_simpleCNN', save_format='h5')
model.load_weights('tf_simpleCNN')


In [ ]:
model.save('tf_simpleCNN', save_format='tf')

# Later, load the model from this directory
model = tf.keras.models.load_model('my_model_saved_model_format')

In [16]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])
model.fit(
  dataflow,
  # validation_data=val_ds,
  steps_per_epoch=1240,
  epochs=1,
)

ValueError: ('Error when checking model input: expected no data, but got:', array([[[[0.5764706 , 0.5529412 , 0.45098042],
         [0.49803925, 0.47450984, 0.37254903],
         [0.6784314 , 0.654902  , 0.5529412 ],
         ...,
         [0.14117648, 0.14117648, 0.13333334],
         [0.14901961, 0.14901961, 0.14117648],
         [0.12941177, 0.12941177, 0.12156864]],

        [[0.6392157 , 0.6039216 , 0.4901961 ],
         [0.7176471 , 0.68235296, 0.5686275 ],
         [0.7607844 , 0.7254902 , 0.6117647 ],
         ...,
         [0.16470589, 0.16470589, 0.15686275],
         [0.12156864, 0.12156864, 0.1137255 ],
         [0.15686275, 0.15686275, 0.14901961]],

        [[0.9725491 , 0.94117653, 0.7960785 ],
         [0.882353  , 0.8470589 , 0.7176471 ],
         [0.79215693, 0.7568628 , 0.6431373 ],
         ...,
         [0.16862746, 0.16862746, 0.16078432],
         [0.16078432, 0.16078432, 0.15294118],
         [0.14117648, 0.14117648, 0.13333334]],

        ...,

        [[0.3529

In [5]:

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
train_model(model, data_flow, epochs=10, learning_rate=0.001)


Start of Epoch 1/10


Seen so far: 0 samples


KeyboardInterrupt: 

In [8]:
!pip install pillow

     ---------------------------------------- 2.5/2.5 MB 2.2 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from datetime import datetime
import os

def pprint(output='\n', show_time=False):
    filename = "hw3-APR18.txt"
    print(output)
    with open(filename, 'a') as f:
        if show_time:
            f.write(datetime.now().strftime("[%Y-%m-%d %H:%M:%S] "))
        f.write(str(output))
        f.write('\n')


In [4]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy, TopKCategoricalAccuracy
import time
import numpy as np

def train(model_lists, model_name, loaders, num_class, phases=['train'], epochs=20, save_weight=False):
    model = model_lists[model_name]()
    
    # Adjust the model based on its name
    if model_name in ["resnet18", "mobileNet_v2"]:
        num_ftrs = model.output_shape[-1]  # Assuming model has been built and output shape is available
        model.layers[-1] = tf.keras.layers.Dense(num_class, activation='softmax')

    pprint(f"Training {model_name}", True)
    
    # Count parameters (sum of all trainable and non-trainable variables)
    model_parameters_amount = np.sum([np.prod(v.shape) for v in model.trainable_variables])
    pprint(f"model total parameters: {model_parameters_amount:,}")

    # Define the optimizer, loss, and metrics
    optimizer = Adam(learning_rate=0.005)
    loss_fn = SparseCategoricalCrossentropy()
    train_acc_metric = SparseCategoricalAccuracy()
    top3_acc_metric = TopKCategoricalAccuracy(k=3)

    pprint(f"learning rate={0.005}")
    start = time.time()

    # Train and validation phases
    for epoch in range(epochs):
        for phase in phases:
            if phase == 'train':
                model.trainable = True
            else:
                model.trainable = False

            running_loss = 0.0
            correct_predictions = 0
            total_samples = 0

            for images, labels in loaders[phase]:
                with tf.GradientTape() as tape:
                    logits = model(images, training=(phase == 'train'))
                    loss = loss_fn(labels, logits)

                if phase == 'train':
                    grads = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(grads, model.trainable_variables))
                
                # Update metrics
                train_acc_metric.update_state(labels, logits)
                top3_acc_metric.update_state(labels, logits)
                
                running_loss += loss.numpy() * images.shape[0]
                total_samples += images.shape[0]

            avg_loss = running_loss / total_samples
            top1_accuracy = train_acc_metric.result().numpy() * 100
            top3_accuracy = top3_acc_metric.result().numpy() * 100

            train_acc_metric.reset_states()
            top3_acc_metric.reset_states()
            
            pprint(f"Epoch [{epoch+1}/{epochs}], phase: {phase}, samples: {total_samples}, Loss: {avg_loss:.4f}, Top-1 Accuracy: {top1_accuracy:.2f}%, Top-3 Accuracy: {top3_accuracy:.2f}%")
    
    duration = time.time() - start
    pprint(f"Elapsed time: {duration} seconds")
    
    if save_weight:
        model.save(f'{model_name}.h5')
        pprint(f"weight saved as: {model_name}.h5")

model_list ={
    "resnet18": lambda: models.resnet18(),
    "mobileNet_v2": lambda: models.mobilenet_v2(),
    "mod_face_a": lambda: mod_resnet(BasicBlock, [2, 2, 0, 0], channel_num_list=[4, 16, 16, ], num_classes=4, input_channel=3),
    "mod_face_b": lambda: mod_resnet(BasicBlock, [2, 2, 0, 0], channel_num_list=[4, 8, 8, ], num_classes=4, input_channel=3),
    "SimpleCNN_a": lambda: SimpleCNN_a(),
    "SimpleCNN_b": lambda: SimpleCNN_b(),
}
model_names = [

    # "resnet18", # 11,178,051
    # "mobileNet_v2", # 2,227,715
    # "mod_face_a", # 17,991
    # "mod_face_b", # 5,199
    # "SimpleCNN_a", # 19,049
    "SimpleCNN_b", # 4,737
]

for ii in range(len(model_names)):
    model_name = model_names[ii]
    phases = ['train', 'val']
    train(model_list, model_name, data_loaders, 4, phases, epochs=1, save_weight=True)

NameError: name 'data_loaders' is not defined

In [17]:
class BasicBlock(Model):
    expansion = 1

    def __init__(self, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = layers.Conv2D(out_channels, kernel_size=3, strides=stride, padding='same', use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(out_channels, kernel_size=3, padding='same', use_bias=False)
        self.bn2 = layers.BatchNormalization()
        self.downsample = downsample
        self.relu = layers.ReLU()

    def call(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out
    
class ModResNet(Model):
    def __init__(self, block, layer_num_list, channel_num_list, num_classes=1000, input_channel=3):
        super(ModResNet, self).__init__()
        self.conv1 = layers.Conv2D(channel_num_list[0], kernel_size=7, strides=2, padding='same', input_shape=(None, None, input_channel), use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.ReLU()
        self.maxpool = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')

        self.layer1 = self._make_layer(block, channel_num_list[1], layer_num_list[0], stride=1)
        self.layer2 = self._make_layer(block, channel_num_list[2], layer_num_list[1], stride=2)
        self.layer3 = self._make_layer(block, channel_num_list[3], layer_num_list[2], stride=2)
        self.layer4 = self._make_layer(block, channel_num_list[4], layer_num_list[3], stride=2)

        self.avgpool = layers.GlobalAveragePooling2D()
        self.fc = layers.Dense(num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride=1):
        downsample = None
        layers = []

        for _ in range(num_blocks):
            layers.append(block(out_channels, stride=stride))
            stride = 1  # Only the first block per layer_group might have stride != 1

        return tf.keras.Sequential(layers)

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)

        x = self.avgpool(x)
        x = self.fc(x)
        return x

# Create a ResNet model instance
model = ModResNet(BasicBlock, [2, 2, 2, 2], [2, 2, 2, 2, 2], num_classes=1000)
# Dummy input tensor
input_tensor = tf.random.normal([1, 224, 224, 3])
# Forward pass
output = model(input_tensor)
print(output.shape)  # Output shape should be (1, 1000) for 1000 classes


ValueError: in converted code:

    C:\Users\User\AppData\Local\Temp\ipykernel_28760\3313432971.py:64 call
        x = self.layer2(x)
    C:\Users\User\AppData\Local\Temp\ipykernel_28760\3313432971.py:26 call
        out += identity
    c:\Users\User\anaconda3\envs\tf37\lib\site-packages\tensorflow\python\ops\math_ops.py:884 binary_op_wrapper
        return func(x, y, name=name)
    c:\Users\User\anaconda3\envs\tf37\lib\site-packages\tensorflow\python\ops\gen_math_ops.py:396 add
        "Add", x=x, y=y, name=name)
    c:\Users\User\anaconda3\envs\tf37\lib\site-packages\tensorflow\python\framework\op_def_library.py:788 _apply_op_helper
        op_def=op_def)
    c:\Users\User\anaconda3\envs\tf37\lib\site-packages\tensorflow\python\util\deprecation.py:507 new_func
        return func(*args, **kwargs)
    c:\Users\User\anaconda3\envs\tf37\lib\site-packages\tensorflow\python\framework\ops.py:3616 create_op
        op_def=op_def)
    c:\Users\User\anaconda3\envs\tf37\lib\site-packages\tenso